设置工作目录

In [ ]:
import os
os.chdir("/KL_PAN_medmnist/")
os.getcwd()

'/content/gdrive/My Drive/H-PAN'

引入辅助函数

In [ ]:

import sys
sys.path.append("/KL_PAN_medmnist/")
import helper


In [ ]:
#from helper import cfg_helper
#from helper import file_helper
#from helper import math_helper
#from helper import pu_learning_dataset
# from helper import ploting_helper

__author__ = 'garrett_local'



In [ ]:
import os
import glob
import time
import pickle
import urllib.request as request
import shutil

import numpy as np

__author__ = 'garrett_local'


def last_modified(path_to_match):
    """
    Get the last modified file among those matching with a given wildcards.
    :param path_to_match: a wildcards indicating what files to find.
    :return: the last modified file among those matching with a given wildcards.
    """
    matched = glob.glob(path_to_match)
    if len(matched) == 0:
        return None
    matched.sort(key=lambda f: os.stat(f).st_mtime)
    newest = matched[-1]
    return newest


def create_dirname_if_not_exist(path):
    create_if_not_exist(os.path.dirname(path))


def create_if_not_exist(path):
    if not os.path.exists(path):
        os.makedirs(path)


def unpickle(file):
    with open(file, 'rb') as fo:
        c = pickle.load(fo, encoding='latin1')
    return c


def download(url, des_path='.'):
    try:
        print('downloading {}'.format(url))
        request.urlretrieve(url, os.path.join(des_path,
                                              os.path.basename(url)))
        request.urlcleanup()
    except request.HTTPError as e:
        print('HTTP Error: {} {}'.format(e.code, url))
    except request.URLError as e:
        print('URL Error: {} {}'.format(e.reason, url))


def get_unique_name():
    return '{}'.format(int(time.time()))


def read_exp_log_data(exp_name):
    path = 'result/{}/log/summaries.pkl'.format(exp_name)
    if os.path.exists(path):
        with open(path, 'rb') as f:
            log_data = pickle.load(f, encoding='latin1')
    else:
        path = 'result/{}/log/summaries.npz'.format(exp_name)
        d = np.load(path)
        log_data = LogData()
        for key in d.keys():
            setattr(log_data, key, d[key])
    return log_data


def save_basetrings_as_text(basestrings, file_name):
    create_dirname_if_not_exist(file_name)
    with open(file_name, 'wb') as f:
        for s in basestrings:
            f.write(s.encode()+b'\n')


def load_basestrings(file_name):
    with open(file_name, 'r') as f:
        basestrings = f.readlines()
    return [s.strip('\n') for s in basestrings]


def save_log_data(log_data, exp_name):
    path = './result/tmp/{}/log'.format(exp_name)
    create_if_not_exist(path)
    with open(os.path.join(path, 'summaries.pkl'), 'wb') as f:
        pickle.dump(log_data, f, protocol=2)


def settle_saved_data(exp_name):
    shutil.move('./result/tmp/{}'.format(exp_name),
                './result/{}'.format(exp_name),)


class LogData(object):
    def __init__(self):
        self.losses = {}

    def __str__(self):
        return '{}'.format(self.__dict__.keys())

    def log_loss(self, key, loss):
        if key not in self.losses:
            self.losses[key] = [loss]
        else:
            self.losses[key].append(loss)


In [ ]:
import configparser


__author__ = 'garrett_local'


def write_cfg_file(path, cfg):
    """
    Write configuration to a txt file.
    :param path: basestring. Indicating where to save the file.
    :param cfg: configuration. Must be a dict containing a few of dict. Each
                dict contained is content of a section. They contain a few
                items, whose value should be basestring.
    """
    conf = configparser.ConfigParser()
    create_dirname_if_not_exist(path)
    cfg_file = open(path, 'w')
    sections = cfg.keys()
    sections = sorted(sections)
    for section in sections:
        content = cfg[section]
        conf.add_section(section)
        items_names = content.keys()
        items_names = sorted(items_names)
        for item_name in items_names:
            conf.set(section, item_name, str(content[item_name]))
    conf.write(cfg_file)
    cfg_file.close()


def write_exp_cfg_file(cfg, exp_name, cfg_name):
    path = './result/tmp/{}/cfg/{}'.format(exp_name, cfg_name)
    write_cfg_file(path, cfg)


def read_cfg_file(path):
    """
    Read configuration saved by function write_cfg_file.
    :param path: basestring. Where to find the configuration file.
    :return: configuration. It is a dict containing a few of dict. Each dict
                contained is content of a section. They contain a few items,
                whose value will be basestring.
    """
    conf = configparser.ConfigParser()
    conf.read(path)
    sections = conf.sections()
    cfg = {}
    if len(sections) == 0:
        raise AssertionError('sections is empty. File {} may not exist or may '
                             'be empty'.format(path))
    for section in sections:
        options = conf.options(section)
        cfg_section = {}
        for option in options:
            item = conf.get(section, option)
            cfg_section[option] = item
        cfg[section] = cfg_section
    return cfg


def to_bool(s):
    if isinstance(s, bool):
        return s
    elif isinstance(s, str):
        return True if s == 'True' else False
    else:
        raise ValueError


In [ ]:
import numpy as np

__author__ = 'garrett_local'


def mask_to_index(mask):
    index = np.array(range(len(mask)))[mask]
    return index


def index_to_mask(index, mask_len):
    mask = np.zeros(mask_len).astype(np.bool)
    mask[index] = True
    return mask


def _crop_common_part(arr1, arr2):
    s1 = arr1.shape
    s2 = arr2.shape
    assert len(s1) == len(s2)
    slc = [slice(0, min(s1[i], s2[i])) for i in range(len(s1))]
    new_mask1 = arr1[slc]
    new_mask2 = arr2[slc]
    return new_mask1, new_mask2


def logical_and(mask1, mask2):
    return np.logical_and(*_crop_common_part(mask1, mask2))


def logical_or(mask1, mask2):
    return np.logical_or(*_crop_common_part(mask1, mask2))


def normalize(arr):
    return arr / np.sum(arr).astype(np.float32)


In [ ]:
import abc
import copy
import math
import numpy as np


__author__ = 'garrett_local'


class DataIterator(object):
    def __init__(self, data_lists, batch_size, max_epoch=None, repeat=True,
                 shuffle=True, epoch_finished=None):
        for idx in range(len(data_lists) - 1):
            assert len(data_lists[idx]) == len(data_lists[idx + 1])
        self._data = data_lists
        self._batch_size = batch_size
        self._repeat = repeat
        self._shuffle = shuffle
        self._num_data = len(self._data[0])
        assert self._num_data >= self._batch_size
        self._shuffle_indexes = self._maybe_generate_shuffled_indexes()
        # 这是用来控制是否重复一套训练，控制阀是检验epoch是否超过设置的最大迭代次数
        self._epoch_finished = 0 if epoch_finished is None else epoch_finished
        self._max_epoch = max_epoch

    @property
    def num_data(self):
        return self._num_data

    @property
    def finished(self):
        if not self._repeat:
            if self.epoch_finished == 1:
                return True
        if self._max_epoch is not None:
            return self.epoch_finished > self._max_epoch
        else:
            return False

    @property
    def epoch_finished(self):
        return self._epoch_finished
    #生成乱序标签
    def _maybe_generate_shuffled_indexes(self):
        indexes = list(range(self._num_data))
        if self._shuffle:
            np.random.shuffle(indexes)
        return indexes

    def get_next_batch(self, batch_size=None):
        if batch_size is None:
            batch_size = self._batch_size
        else:
            assert self._num_data >= batch_size
        #定义一个异常，用raise 抛出StopIteration()
        if len(self._shuffle_indexes) == 0:
            raise StopIteration()
        if len(self._shuffle_indexes) >= batch_size:  # when data left is enough
            indexes = self._shuffle_indexes[:batch_size]
            #这一步是为了下一次迭代做准备，把已经用过的数据抛弃掉
            self._shuffle_indexes = self._shuffle_indexes[batch_size:]
        else:  # when data left is not enough.
            indexes = self._shuffle_indexes
            self._shuffle_indexes = []
        # 一次训练是否完成的条件是：剩下的数据量小于给定的批次量。
        #当一次训练完成后，需要通过随机打乱函数重新生成随机的索引
        if len(self._shuffle_indexes) == 0:
            self._epoch_finished += 1
            if self._repeat:
                if self._max_epoch is not None:
                    if self._epoch_finished > self._max_epoch:
                        raise StopIteration()
                self._shuffle_indexes = self._maybe_generate_shuffled_indexes()
                # 当数据量不够一个批次的训练时，需要从self._shuffle_indexes的前面开始截取缺少的部分
                num_left = batch_size - len(indexes)
                #extend () 函数用于在列表末尾一次性追加另一个序列中的多个值（用新列表扩展原来的列表）
                indexes.extend(self._shuffle_indexes[:num_left])
                #截取过的数据不再使用
                self._shuffle_indexes = self._shuffle_indexes[num_left:]
        return [l[indexes] for l in self._data]

    def __iter__(self):
        return self

    def __next__(self):
        return self.get_next_batch()


class PuDataIterator(object):
    def __init__(self, u_data, l_data, batch_size, max_epoch=None,
                 epoch_finished=0, repeat=True, shuffle=True):
        #the number of labeled and unlabeled data
        self._u_num = u_data[0].shape[0]
        self._l_num = l_data[0].shape[0]
        self._data_num = self._u_num + self._l_num
        self._p_u = float(self._u_num) / float(self._u_num + self._l_num)
        self._p_l = float(self._l_num) / float(self._u_num + self._l_num)
        self._batch_size = batch_size
        self._used_u_num, self._used_l_num = 0, 0
        self._u_iterator = DataIterator(u_data, int(batch_size * self._p_u),
                                        repeat=repeat, shuffle=shuffle,
                                        epoch_finished=epoch_finished,
                                        max_epoch=max_epoch)
        self._l_iterator = DataIterator(l_data, int(batch_size * self._p_l),
                                        repeat=repeat, shuffle=shuffle,
                                        epoch_finished=epoch_finished,
                                        max_epoch=max_epoch)
        self._finished_epoch = epoch_finished
        self._max_epoch = max_epoch
        self._repeat = repeat
        self._shuffle = shuffle
        #这是迭代次数吗？
        self._len = int(self._data_num / self._batch_size)

    @property
    def epoch_finished(self):
        return self._finished_epoch

    @property
    def num_data(self):
        return self._data_num

    @property
    def finished(self):
        if self._max_epoch is not None:
            return self.epoch_finished > self._max_epoch
        else:
            return False

    def __next__(self):
        used_num = self._used_l_num + self._used_u_num + self._batch_size
        #round()是python自带的一个函数，用于数字的四舍五入
        next_u_num = round(used_num * self._p_u - self._used_u_num)
        self._used_u_num += next_u_num
        next_l_num = round(used_num * self._p_l - self._used_l_num)
        next_l_num += self._batch_size - next_u_num - next_l_num
        self._used_l_num += next_l_num
        # Whatever the case, at least one sample is expected from each iterator
        # (though the iterator may be empty, when self._repeat == False).
        assert next_l_num != 0 and next_u_num != 0

        if self._max_epoch is not None:
            if self._finished_epoch >= self._max_epoch:
                raise StopIteration()
        # Stop iteration only if both iterator is finished. So no data will
        # be missed.
        if self._u_iterator.finished and self._l_iterator.finished:
            raise StopIteration()
        #用except捕捉异常StopIteration，然后令u_data=0
        try:
            u_data = self._u_iterator.get_next_batch(int(next_u_num))
        except StopIteration:
            u_data = None

        try:
            l_data = self._l_iterator.get_next_batch(int(next_l_num))
        except StopIteration:
            l_data = None

        if not self._repeat:

            # It is guaranteed here that, if one of the iterator is finished,
            # another one will make up the missing part.
            if self._u_iterator.finished and not self._l_iterator.finished:
                u_num = 0 if u_data is None else u_data[0].shape[0]
                left = self._l_iterator.get_next_batch(int(next_u_num - u_num))
                l_data = [np.concatenate((l_data[i], left[i]))
                          for i in range(len(l_data))]
            if self._l_iterator.finished and not self._u_iterator.finished:
                l_num = 0 if l_data is None else l_data[0].shape[0]
                left = self._u_iterator.get_next_batch(int(next_l_num - l_num))
                u_data = [np.concatenate((u_data[i], left[i]))
                          for i in range(len(u_data))]

        self._finished_epoch = min(self._u_iterator.epoch_finished,
                                   self._l_iterator.epoch_finished)
        if u_data is None:
            return l_data
        elif l_data is None:
            return u_data
        else:
            # print(len(l_data))
            return [np.concatenate((u_data[i], l_data[i]))
                    for i in range(len(l_data))]
            # return [(u_data[i], l_data[i])
            # for i in range(len(l_data))]

    def __iter__(self):
        return self


class PuLearningDataSet(object):
    def __init__(self, cfg):
        self._cfg = cfg
        self._num_labeled = int(self._cfg['num_labeled'])
        self.__num_unlabeled = None
        self._overlap = to_bool(self._cfg['overlap'])
        self._max_epoch = int(self._cfg['max_epoch'])
        self._batch_size = int(self._cfg['batch_size'])
        self._prior = None
        #shuffle在机器学习与深度学习中代表的意思是，将训练模型的数据集进行打乱的操作
        self._shuffled_indexes = None  # shuffle indexes for positive-unlabeled division.
        self._negative = 0
        self._positive = 1

        self._unlabeled_positive_mask = None
        self._unlabeled_negative_mask = None
        self._labeled_positive_mask = None
        self._labeled_negative_mask = None

        # for initialize member variables.
        self._prepare_pu_training_data()
    # @property是python的一种装饰器，是用来修饰方法的，我们可以使用@property装饰器来创建只读属性，
    # @property装饰器会将方法转换为相同名称的只读属性,可以与所定义的属性配合使用，这样可以防止属性被修改
    @property
    def _num_unlabeled(self):
        assert self.__num_unlabeled is not None
        return self.__num_unlabeled

    @property
    def batch_size(self):
        return self._batch_size

    def _prepare_pu_training_data(self):
        positive = self._positive
        negative = self._negative
        train_y = self._original_train_y()
        train_x = self._original_train_x()
        if self._shuffled_indexes is None:
            #创建一个0-len（标签）长度的数组
            self._shuffled_indexes = np.array(range(len(train_y)))
            #通过 random 静态对象调用该方法，shuffle直接在原来的数组上进行操作，改变原来数组的顺序，无返回值
            np.random.shuffle(self._shuffled_indexes)
        assert self._shuffled_indexes.shape[0] == len(train_y)
        train_y = train_y[self._shuffled_indexes]
        train_x = train_x[self._shuffled_indexes]
        #copy.deepcopy () 函数是一个深复制函数。 所谓深复制 ，就是从输入变量完全复刻一个相同的变量，无论怎么改变新变量，原有变量的值都 不会受到影响
        true_y = copy.deepcopy(train_y)
        num_pos = (train_y == positive).sum()
        num_neg = (train_y == negative).sum()
        #如何优交集
        if self._overlap:
            self.__num_unlabeled = num_neg + num_pos
            self._prior = float(num_pos) / float(num_neg + num_pos)

            #mask to index, samples with a number of _num_labeled is selected
            #创建一个掩码，通过切片操作到1000
            overlapped_indexes = mask_to_index(
                train_y == positive
            )[:self._num_labeled]

            train_x_labeled = train_x[overlapped_indexes]
            true_y_labeled = true_y[overlapped_indexes]
            train_x = np.concatenate((train_x, train_x_labeled), axis=0)
            true_y = np.concatenate((true_y, true_y_labeled), axis=0)
            train_y = np.concatenate(
                (negative * np.ones(train_y.shape), np.ones(self._num_labeled)),
            )
        else:
            self.__num_unlabeled = num_neg + num_pos - self._num_labeled
            self._prior = \
                float(num_pos - self._num_labeled) / \
                float(num_neg + num_pos - self._num_labeled)
            train_y[train_y == positive][self._num_labeled:] = negative
        #train_y == negative denote for unlabeled data, while true_y == positive denote for positive samples,thus and operation
        self._unlabeled_positive_mask = np.logical_and(train_y == negative,
                                                       true_y == positive)
        self._unlabeled_negative_mask = np.logical_and(train_y == negative,
                                                       true_y == negative)
        self._labeled_positive_mask = np.logical_and(train_y == positive,
                                                     true_y == positive)
        self._labeled_negative_mask = np.logical_and(train_y == positive,
                                                     true_y == negative)
        return train_x, train_y

    def _prepare_pn_testing_data(self):
        #just return original test data as the pn test data
        test_y = self._original_test_y()
        test_x = self._original_test_x()
        #把test_y调成1列的结构
        test_y = test_y.reshape([-1, 1])
        return test_x, test_y

    def get_training_iterator(self, batch_size=None, repeat=True, shuffle=True,
                              max_epoch=None):
        x, y = self._prepare_pu_training_data()
        if max_epoch is None:
            max_epoch = self._max_epoch
        if batch_size is None:
            batch_size = self._batch_size
        y = y.reshape([-1, 1])
        #这是一个或的结构，self._unlabeled_positive_mask和self._unlabeled_negative_mask里真的元素加到一起
        unlabeled_mask = np.logical_or(self._unlabeled_positive_mask,
                                       self._unlabeled_negative_mask)
        u_x = x[unlabeled_mask]
        u_y = y[unlabeled_mask]

        labeled_mask = np.logical_or(self._labeled_positive_mask,
                                     self._labeled_negative_mask)
        l_x = x[labeled_mask]
        l_y = y[labeled_mask]

        return PuDataIterator((u_x, u_y),
                              (l_x, l_y),
                              batch_size, max_epoch=max_epoch, repeat=repeat,
                              shuffle=shuffle)

    def get_testing_iterator(self, batch_size=None):
        x, y = self._prepare_pn_testing_data()
        if batch_size is None:
            batch_size = \
                self._batch_size if self._batch_size <= len(y) else len(y)
        return DataIterator((x, y), batch_size, max_epoch=1, repeat=False,
                            shuffle=False)

    @property
    def prior(self):
        assert self._prior is not None
        return self._prior

    @abc.abstractmethod
    def _original_train_x(self):
        pass
    # 我们抽象出一个基类，知道要有哪些方法，但只是抽象方法，并不实现功能，只能继承，而不能被实例化，
    # 但子类必须要实现该方法，这就需要用到抽象基类
    @abc.abstractmethod
    def _original_train_y(self):
        """
        Should return  binarized labels.
        """
        pass

    @abc.abstractmethod
    def _original_test_x(self):
        pass

    @abc.abstractmethod
    def _original_test_y(self):
        """
        Should return  binarized labels.
        """
        pass


class PnLearningDataSet(object):
    def __init__(self, cfg, prior):
        self._cfg = cfg
        self._max_epoch = int(self._cfg['max_epoch'])
        self._batch_size = int(self._cfg['batch_size'])
        self._prior = prior
        self._shuffled_indexes = None  # shuffle indexes for positive-unlabeled division.
        if prior is None:
            self._num_pos = None
            self._num_neg = None
        else:
            self._num_pos = int(self._cfg['num_pos'])
            self._num_neg = int(math.pow((1 - prior) / (2 * prior), 2) * self._num_pos)

    @property
    def batch_size(self):
        return self._batch_size

    def _prepare_pn_training_data(self):
        positive = 1
        negative = -1
        train_y = self._original_train_y()
        train_x = self._original_train_x()
        if self._shuffled_indexes is None:
            self._shuffled_indexes = list(range(len(train_y)))
            np.random.shuffle(self._shuffled_indexes)
        train_y = train_y[self._shuffled_indexes]
        train_x = train_x[self._shuffled_indexes]
        num_pos = (train_y == positive).sum()
        num_neg = (train_y == negative).sum()
        if self._num_neg is None:
            self._num_neg = num_neg
        if self._num_pos is None:
            self._num_pos = num_pos
        assert num_pos >= self._num_pos and num_neg >= self._num_neg
        train_x = np.concatenate((train_x[train_y == positive][:self._num_pos],
                                 train_x[train_y == negative][:self._num_neg]))
        train_y = np.concatenate((train_y[train_y == positive][:self._num_pos],
                                 train_y[train_y == negative][:self._num_neg]))
        train_y = train_y.reshape([-1, 1])
        return train_x, train_y

    def _prepare_pn_testing_data(self):
        test_y = self._original_test_y()
        test_x = self._original_test_x()
        test_y = test_y.reshape([-1, 1])
        return test_x, test_y

    def get_training_iterator(self, batch_size=None, repeat=True, shuffle=True,
                              max_epoch=None):
        x, y = self._prepare_pn_training_data()
        if max_epoch is None:
            max_epoch = self._max_epoch
        if batch_size is None:
            batch_size = \
                self._batch_size if self._batch_size <= len(y) else len(y)
        return DataIterator((x, y), batch_size, max_epoch=max_epoch,
                            repeat=repeat, shuffle=shuffle)

    def get_testing_iterator(self, batch_size=None):
        x, y = self._prepare_pn_testing_data()
        if batch_size is None:
            batch_size = \
                self._batch_size if self._batch_size <= len(y) else len(y)
        return DataIterator((x, y), batch_size, max_epoch=1, repeat=False,
                            shuffle=False)

    @property
    def prior(self):
        assert self._prior is not None
        return self._prior

    @abc.abstractmethod
    def _original_train_x(self):
        pass

    @abc.abstractmethod
    def _original_train_y(self):
        """
        Should return  binarized labels.
        """
        pass

    @abc.abstractmethod
    def _original_test_x(self):
        pass

    @abc.abstractmethod
    def _original_test_y(self):
        """
        Should return  binarized labels.
        """
        pass


引入数据集并进行处理

In [ ]:
!pip install medmnist
import os
import numpy as np
from PIL import Image
from torch.utils.data import Dataset
from medmnist.info import INFO, HOMEPAGE, DEFAULT_ROOT

In [ ]:
import os
import numpy as np
from PIL import Image
from torch.utils.data import Dataset
from medmnist.info import INFO, HOMEPAGE, DEFAULT_ROOT


class MedMNIST(Dataset):

    flag = ...

    def __init__(self,
                 split,
                 transform=None,
                 target_transform=None,
                 download=False,
                 as_rgb=False,
                 root=DEFAULT_ROOT):
        ''' dataset
        :param split: 'train', 'val' or 'test', select subset
        :param transform: data transformation
        :param target_transform: target transformation
        '''

        self.info = INFO[self.flag]

        if root is not None and os.path.exists(root):
            self.root = root
        else:
            raise RuntimeError("Failed to setup the default `root` directory. " +
                               "Please specify and create the `root` directory manually.")

        if download:
            self.download()

        if not os.path.exists(
                os.path.join(self.root, "{}.npz".format(self.flag))):
            raise RuntimeError('Dataset not found. ' +
                               ' You can set `download=True` to download it')

        npz_file = np.load(os.path.join(self.root, "{}.npz".format(self.flag)))

        self.split = split
        self.transform = transform
        self.target_transform = target_transform
        self.as_rgb = as_rgb

        if self.split == 'train':
            self.imgs = npz_file['train_images']
            self.labels = npz_file['train_labels']
        elif self.split == 'val':
            self.imgs = npz_file['val_images']
            self.labels = npz_file['val_labels']
        elif self.split == 'test':
            self.imgs = npz_file['test_images']
            self.labels = npz_file['test_labels']
        else:
            raise ValueError

    def __len__(self):
        return self.imgs.shape[0]

    def __repr__(self):
        '''Adapted from torchvision.ss'''
        _repr_indent = 4
        head = f"Dataset {self.__class__.__name__} ({self.flag})"
        body = [f"Number of datapoints: {self.__len__()}"]
        body.append(f"Root location: {self.root}")
        body.append(f"Split: {self.split}")
        body.append(f"Task: {self.info['task']}")
        body.append(f"Number of channels: {self.info['n_channels']}")
        body.append(f"Meaning of labels: {self.info['label']}")
        body.append(f"Number of samples: {self.info['n_samples']}")
        body.append(f"Description: {self.info['description']}")
        body.append(f"License: {self.info['license']}")

        lines = [head] + [" " * _repr_indent + line for line in body]
        return '\n'.join(lines)

    def download(self):
        try:
            from torchvision.datasets.utils import download_url
            download_url(url=self.info["url"],
                         root=self.root,
                         filename="{}.npz".format(self.flag),
                         md5=self.info["MD5"])
        except:
            raise RuntimeError('Something went wrong when downloading! ' +
                               'Go to the homepage to download manually. ' +
                               HOMEPAGE)


class MedMNIST2D(MedMNIST):

    def __getitem__(self, index):
        '''
        return: (without transform/target_transofrm)
            img: PIL.Image
            target: np.array of `L` (L=1 for single-label)
        '''
        img, target = self.imgs[index], self.labels[index].astype(int)
        img = Image.fromarray(img)

        if self.as_rgb:
            img = img.convert('RGB')

        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target

    def save(self, folder, postfix="png", write_csv=True):

        from medmnist.utils import save2d

        save2d(imgs=self.imgs,
               labels=self.labels,
               img_folder=os.path.join(folder, self.flag),
               split=self.split,
               postfix=postfix,
               csv_path=os.path.join(folder, f"{self.flag}.csv") if write_csv else None)

    def montage(self, length=20, replace=False, save_folder=None):
        from medmnist.utils import montage2d

        n_sel = length * length
        sel = np.random.choice(self.__len__(), size=n_sel, replace=replace)

        montage_img = montage2d(imgs=self.imgs,
                                n_channels=self.info['n_channels'],
                                sel=sel)

        if save_folder is not None:
            if not os.path.exists(save_folder):
                os.makedirs(save_folder)
            montage_img.save(os.path.join(save_folder,
                                          f"{self.flag}_{self.split}_montage.jpg"))

        return montage_img

class BreastMNIST(MedMNIST2D):
    flag = "breastmnist"
class PneumoniaMNIST(MedMNIST2D):
    flag = "pneumoniamnist"
class OCTMNIST(MedMNIST2D):
    flag = "octmnist"
class BloodMNIST(MedMNIST2D):
    flag = "bloodmnist"

引入乳腺超声数据集

In [ ]:
__author__ = 'garrett_local'


def _prepare_BreastMNIST_data():
    train_x = []
    train_y = []
    train_x1 = BreastMNIST("train",download=True).imgs
    train_y1 = BreastMNIST("train",download=True).labels
    train_x2 = BreastMNIST("val",download=True).imgs
    train_y2 = BreastMNIST("val",download=True).labels
    train_x = np.concatenate((train_x1,train_x2),axis=0)
    train_y = np.concatenate((train_y1,train_y2),axis=0)
    test_x = BreastMNIST("test",download=True).imgs
    test_y = BreastMNIST("test",download=True).labels

    train_x = np.reshape(np.array(train_x), (train_x.shape[0], 28, 28, 1)) / 255.
    test_x = np.reshape(np.array(test_x), (test_x.shape[0], 28, 28, 1)) / 255.

    train_x = np.asarray(train_x[:], dtype=np.float32)
    train_y = np.asarray(train_y[:], dtype=np.int32).squeeze(1)

    test_x = np.asarray(test_x[:], dtype=np.float32)
    test_y = np.asarray(test_y[:], dtype=np.int32).squeeze(1)
    # pdb.set_trace()
    return train_x, train_y, test_x, test_y
class BreastMNISTDataset(PuLearningDataSet):

    def __init__(self, *args, **kwargs):
        self._train_x, self._train_y, self._test_x, self._test_y = \
            _prepare_BreastMNIST_data()
        super(BreastMNISTDataset, self).__init__(*args, **kwargs)

    def _original_train_x(self):
        return self._train_x

    def _original_train_y(self):
        return self._train_y

    def _original_test_x(self):
        return self._test_x

    def _original_test_y(self):
        return self._test_y


class BreastMNISTPnDataset(PnLearningDataSet):

    def __init__(self, *args, **kwargs):
        self._train_x, self._train_y, self._test_x, self._test_y = \
            _prepare_BreastMNIST_data()
        super(BreastMNISTPnDataset, self).__init__(*args, **kwargs)

    def _original_train_x(self):
        return self._train_x

    def _original_train_y(self):
        return self._train_y

    def _original_test_x(self):
        return self._test_x

    def _original_test_y(self):
        return self._test_y

引入胸部X光数据集

In [ ]:
def _prepare_PneumoniaMNIST_data():
    train_x = []
    train_y = []
    train_x1 = PneumoniaMNIST("train",download=True).imgs
    train_y1 = PneumoniaMNIST("train",download=True).labels
    train_x2 = PneumoniaMNIST("val",download=True).imgs
    train_y2 = PneumoniaMNIST("val",download=True).labels
    train_x = np.concatenate((train_x1,train_x2),axis=0)
    train_y = np.concatenate((train_y1,train_y2),axis=0)
    test_x = PneumoniaMNIST("test",download=True).imgs
    test_y = PneumoniaMNIST("test",download=True).labels

    train_x = np.reshape(np.array(train_x), (train_x.shape[0], 28, 28, 1)) / 255.
    test_x = np.reshape(np.array(test_x), (test_x.shape[0], 28, 28, 1)) / 255.

    train_x = np.asarray(train_x[:], dtype=np.float32)
    train_y = np.asarray(train_y[:], dtype=np.int32).squeeze(1)

    test_x = np.asarray(test_x[:], dtype=np.float32)
    test_y = np.asarray(test_y[:], dtype=np.int32).squeeze(1)
    # pdb.set_trace()
    return train_x, train_y, test_x, test_y

class PneumoniaMNISTDataset(PuLearningDataSet):

    def __init__(self, *args, **kwargs):
        self._train_x, self._train_y, self._test_x, self._test_y = \
            _prepare_PneumoniaMNIST_data()
        super(PneumoniaMNISTDataset, self).__init__(*args, **kwargs)

    def _original_train_x(self):
        return self._train_x

    def _original_train_y(self):
        return self._train_y

    def _original_test_x(self):
        return self._test_x

    def _original_test_y(self):
        return self._test_y


class PneumoniaMNISTPnDataset(PnLearningDataSet):

    def __init__(self, *args, **kwargs):
        self._train_x, self._train_y, self._test_x, self._test_y = \
            _prepare_PneumoniaMNIST_data()
        super(PneumoniaMNISTPnDataset, self).__init__(*args, **kwargs)

    def _original_train_x(self):
        return self._train_x

    def _original_train_y(self):
        return self._train_y

    def _original_test_x(self):
        return self._test_x

    def _original_test_y(self):
        return self._test_y

引入OCT数据集

In [ ]:
def _prepare_OCTMNIST_data():
    train_x = []
    train_y = []
    train_x1 = OCTMNIST("train",download=True).imgs
    train_y1 = OCTMNIST("train",download=True).labels
    train_x2 = OCTMNIST("val",download=True).imgs
    train_y2 = OCTMNIST("val",download=True).labels
    train_x = np.concatenate((train_x1,train_x2),axis=0)
    train_y = np.concatenate((train_y1,train_y2),axis=0)
    test_x = OCTMNIST("test",download=True).imgs
    test_y = OCTMNIST("test",download=True).labels

    train_x = np.reshape(np.array(train_x), (train_x.shape[0], 28, 28, 1)) / 255.
    test_x = np.reshape(np.array(test_x), (test_x.shape[0], 28, 28, 1)) / 255.

    train_x = np.asarray(train_x[:], dtype=np.float32)
    train_y = np.asarray(train_y[:], dtype=np.int32).squeeze(1)

    test_x = np.asarray(test_x[:], dtype=np.float32)
    test_y = np.asarray(test_y[:], dtype=np.int32).squeeze(1)

    # Binarize labels.
    train_y[train_y % 2 == 1] = -1
    train_y[train_y % 2 == 0] = 1
    train_y[train_y == -1] = 0
    test_y[test_y % 2 == 1] = -1
    test_y[test_y % 2 == 0] = 1
    test_y[test_y == -1] = 0
    return train_x, train_y, test_x, test_y

class OCTMNISTDataset(PuLearningDataSet):

    def __init__(self, *args, **kwargs):
        self._train_x, self._train_y, self._test_x, self._test_y = \
            _prepare_OCTMNIST_data()
        super(OCTMNISTDataset, self).__init__(*args, **kwargs)

    def _original_train_x(self):
        return self._train_x

    def _original_train_y(self):
        return self._train_y

    def _original_test_x(self):
        return self._test_x

    def _original_test_y(self):
        return self._test_y


class OCTMNISTPnDataset(PnLearningDataSet):

    def __init__(self, *args, **kwargs):
        self._train_x, self._train_y, self._test_x, self._test_y = \
            _prepare_OCTMNIST_data()
        super(OCTMNISTPnDataset, self).__init__(*args, **kwargs)

    def _original_train_x(self):
        return self._train_x

    def _original_train_y(self):
        return self._train_y

    def _original_test_x(self):
        return self._test_x

    def _original_test_y(self):
        return self._test_y

引入BloodMNIST数据集

In [ ]:
def _prepare_BloodMNIST_data():
    train_x = []
    train_y = []
    train_x1 = BloodMNIST("train",download=True).imgs
    train_y1 = BloodMNIST("train",download=True).labels
    train_x2 = BloodMNIST("val",download=True).imgs
    train_y2 = BloodMNIST("val",download=True).labels
    train_x = np.concatenate((train_x1,train_x2),axis=0)
    train_y = np.concatenate((train_y1,train_y2),axis=0)
    test_x = BloodMNIST("test",download=True).imgs
    test_y = BloodMNIST("test",download=True).labels

    train_x = train_x.reshape((train_x.shape[0], 3, 28, 28)) / 255.0 #transpose([0, 2, 3, 1])
    test_x = test_x.reshape((test_x.shape[0], 3, 28, 28)) / 255.0 #.transpose([0, 2, 3, 1])

    train_x = np.asarray(train_x[:], dtype=np.float32)
    train_y = np.asarray(train_y[:], dtype=np.int32).squeeze(1)

    test_x = np.asarray(test_x[:], dtype=np.float32)
    test_y = np.asarray(test_y[:], dtype=np.int32).squeeze(1)

    # Binarize labels.
    train_y[train_y % 2 == 1] = -1
    train_y[train_y % 2 == 0] = 1
    train_y[train_y == -1] = 0
    test_y[test_y % 2 == 1] = -1
    test_y[test_y % 2 == 0] = 1
    test_y[test_y == -1] = 0
    return train_x, train_y, test_x, test_y

class BloodMNISTDataset(PuLearningDataSet):

    def __init__(self, *args, **kwargs):
        self._train_x, self._train_y, self._test_x, self._test_y = \
            _prepare_BloodMNIST_data()
        super(BloodMNISTDataset, self).__init__(*args, **kwargs)

    def _original_train_x(self):
        return self._train_x

    def _original_train_y(self):
        return self._train_y

    def _original_test_x(self):
        return self._test_x

    def _original_test_y(self):
        return self._test_y


class BloodMNISTPnDataset(PnLearningDataSet):

    def __init__(self, *args, **kwargs):
        self._train_x, self._train_y, self._test_x, self._test_y = \
            _prepare_BloodMNIST_data()
        super(BloodMNISTPnDataset, self).__init__(*args, **kwargs)

    def _original_train_x(self):
        return self._train_x

    def _original_train_y(self):
        return self._train_y

    def _original_test_x(self):
        return self._test_x

    def _original_test_y(self):
        return self._test_y

In [ ]:
#from helper import cifar10_dataset
#from helper import mnist_dataset
#from helper import sddd_dataset
#from helper import news_dataset
# import network

__author__ = 'garrett_local'


def load_dataset(cfg):
    dataset_name = cfg['dataset']['dataset_name']

    if dataset_name == 'BreastMNIST':
        return BreastMNISTDataset, BreastMNISTPnDataset
    if dataset_name == 'PneumoniaMNIST':
        return PneumoniaMNISTDataset, PneumoniaMNISTPnDataset
    if dataset_name == 'OCTMNIST':
        return OCTMNISTDataset, OCTMNISTPnDataset
    if dataset_name == 'BloodMNIST':
        return BloodMNISTDataset, BloodMNISTPnDataset
    if dataset_name == 'AMD':
        return AMDDataset, AMDPnDataset



from helper import ploting_helper 失败，复制源代码

引入模块库


In [ ]:
import argparse
import os
import numpy as np
import math

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable


import torch.nn as nn
import torch.nn.functional as F
import torch
import pdb

设置参数

parser.add_argument('--model', type=str, default='pan', choices=['pan', 'nnpu', 'upu', 'agan'],
                    help='Please select model that you want to run.')
parser.add_argument('--dataset', type=str, default='cifar10', choices=['mnist', 'cifar10', 'sddd', 'news'],
                    help='Using dataset.')

In [ ]:
class Argument():
    def __init__(self):
      self.n_epochs=200
      self.batch_size=64
      self.dropoutrate=0.3
      self.lr=0.0001
      self.b1=0.5
      self.b2=0.999
      self.n_cpu=8
      self.latent_dim=100
      self.img_size=28
      self.channels=1
      self.sample_interval=400
      self.gpu=0
      self.model='pan'
      self.dataset='BloodMNIST'

In [ ]:
os.makedirs('images', exist_ok=True)
opt = Argument()
#print(opt.channels, opt.img_size, opt.img_size)
model_style = {'BreastMNIST': 'mlp', 'PneumoniaMNIST': 'mlp', 'OCTMNIST': 'mlp', 'BloodMNIST': 'conv','AMD': 'conv'}
img_shape = (opt.channels, opt.img_size, opt.img_size)
img_shape = (1, 28, 28)
#用默认的CPU
torch.cuda.set_device(0)
cuda = True if torch.cuda.is_available() else False
#给定学习率，目标函数，
ratios = [0.8, 0.7, 0.6, 0.5, 0.4]
#目标函数  KL散度
optimize_type = "holder"
loss_type = "3rd"
# ratio = 0.6
#ratios = [0.5,0.4]

主函数

In [ ]:
for ratio in ratios:
    def adjust_lr(optimizer, epoch):
        lr = opt.lr
        #“//”:取整除 - 返回商的整数部分（向下取整）
        # “**”:幂运算
        lr1 = opt.lr * (0.5 ** (epoch // 20))
        if lr == lr1:
            return
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr1


    class Discriminator_MLP(nn.Module):
        def __init__(self):
            super(Discriminator_MLP, self).__init__()
            # dense1_bn = nn.BatchNorm1d(512)
            # dense2_bn = nn.BatchNorm1d(256)
            #创建一个MLP全连接模型，输入是784个像素点，一共10层,
            #第一层也就是序号为0的层数，输入是784个像素点，输出是512个像素点
            #第二层是归一化处理，BatchNorm通过对加入参数后的数据进行归一处理，来加快收敛速度和防止过拟合
            #第三层是为了解决Dead ReLU问题的激活函数层
            #第四次是Dropout，是用来防止过拟合的。
            # nn.Dropout(p = 0.3) 表示每个神经元有0.3的可能性不被激活
            self.model = nn.Sequential(
                nn.Linear(int(np.prod(img_shape)), 512),
                nn.BatchNorm1d(512),
                nn.LeakyReLU(0.2, inplace=True),
                nn.Dropout(opt.dropoutrate),
                nn.Linear(512, 256),
                nn.BatchNorm1d(256),
                nn.LeakyReLU(0.2, inplace=True),
                nn.Dropout(opt.dropoutrate),
                nn.Linear(256, 1),
                nn.Sigmoid()
            )

        def forward(self, img):
            #.view(x.size(0), -1)这句话是说将第二次卷积的输出拉伸为一行，这句代码中的上一句中x的执行结果为：50*32*7*7个数
            # 其中，50代表的是批次训练是选取的批量数BATCH_SIZE，每次选择50个数进行训练。
            # 32代表的是out_channels，7*7代表的是每张图像处理之后的尺度。
            img_flat = img.view(img.size(0), -1)
            validity = self.model(img_flat)

            return validity


    class Recognizer_MLP(nn.Module):
        def __init__(self):
            super(Recognizer_MLP, self).__init__()
            # dense1_bn = nn.BatchNorm1d(512)
            # dense2_bn = nn.BatchNorm1d(256)

            self.model = nn.Sequential(
                nn.Linear(int(np.prod(img_shape)), 512),
                nn.BatchNorm1d(512),
                # nn.ReLU(),
                nn.LeakyReLU(0.2, inplace=True),
                nn.Dropout(opt.dropoutrate),
                nn.Linear(512, 256),
                nn.BatchNorm1d(256),
                # nn.ReLU(),
                nn.LeakyReLU(0.2, inplace=True),
                nn.Dropout(opt.dropoutrate),
                nn.Linear(256, 1),
                nn.Sigmoid()
            )

        def forward(self, img):
            img_flat = img.view(img.size(0), -1)
            validity = self.model(img_flat)

            return validity


    class Recognizer_CNN(nn.Module):
        def __init__(self):
            super(Recognizer_CNN, self).__init__()
            # dense1_bn = nn.BatchNorm1d(512)
            # dense2_bn = nn.BatchNorm1d(256)
            #创建一个15层的卷积神经网络
            #第一层是输出频道数为3，输入频道数为96，卷积核是3*3，其他为默认值
            #输入32*32的RGB格式的图像，输出的是这个尺寸:torch.Size([1, 10, 14, 14])
            self.conv = nn.Sequential(
                nn.Conv2d(3, 84, 3),
                nn.BatchNorm2d(84),
                nn.ReLU(),
                # nn.BatchNorm2d(84),
                nn.Dropout(opt.dropoutrate),
                # nn.LeakyReLU(0.2, inplace=True),
                nn.Conv2d(84, 84, 3, stride=2),
                nn.BatchNorm2d(84),
                nn.ReLU(),
                # nn.BatchNorm2d(84),
                nn.Dropout(opt.dropoutrate),
                # nn.LeakyReLU(0.2, inplace=True),
                nn.Conv2d(84, 168, 1),
                nn.BatchNorm2d(168),
                nn.ReLU(),
                # nn.BatchNorm2d(168),
                nn.Dropout(opt.dropoutrate),
                # nn.LeakyReLU(0.2, inplace=True),
                nn.Conv2d(168, 8, 1),
                nn.ReLU(),
                # nn.BatchNorm2d(8),
                nn.Dropout(opt.dropoutrate),
                # nn.LeakyReLU(0.2, inplace=True),
            )

            self.fc1 = nn.Sequential(
                nn.Linear(144 * 8, 1000),
                nn.LeakyReLU(0.2, inplace=True),
                # nn.Dropout(opt.dropoutrate),
                nn.Linear(1000, 1),
                nn.Sigmoid(),
            )

        def forward(self, img):
            conv_d = self.conv(img)
            out = self.fc1(conv_d.view(conv_d.shape[0], -1))

            return out


    class Discriminator_CNN(nn.Module):
        def __init__(self):
            super(Discriminator_CNN, self).__init__()
            # dense1_bn = nn.BatchNorm1d(512)
            # dense2_bn = nn.BatchNorm1d(256)

            self.conv = nn.Sequential(
                nn.Conv2d(3, 84, 3),
                nn.ReLU(),
                nn.BatchNorm2d(84),
                # nn.LeakyReLU(0.2, inplace=True),
                nn.Dropout(opt.dropoutrate),
                nn.Conv2d(84, 84, 3, stride=2),
                # nn.BatchNorm2d(84),
                nn.ReLU(),
                nn.BatchNorm2d(84),
                nn.Dropout(opt.dropoutrate),
                nn.Conv2d(84, 168, 1),
                nn.ReLU(),
                nn.BatchNorm2d(168),
                nn.Dropout(opt.dropoutrate),
                # nn.BatchNorm2d(84),
                # nn.LeakyReLU(0.2, inplace=True),
                nn.Conv2d(168, 8, 1),
                nn.ReLU(),
                # nn.BatchNorm2d(8),
                # nn.LeakyReLU(0.2, inplace=True),
            )

            self.fc1 = nn.Sequential(
                nn.Linear(144 * 8, 1000),
                # nn.ReLU(),
                nn.LeakyReLU(0.2, inplace=True),
                # nn.Dropout(opt.dropoutrate),
                nn.Linear(1000, 1),
                nn.Sigmoid(),
            )

        def forward(self, img):
            conv_d = self.conv(img)
            #view 重新定义矩阵形式
            out = self.fc1(conv_d.view(conv_d.shape[0], -1))

            return out

    #将得分结果传入这个模型里，将得到的索引转换为标签
    def index2label(index):
        return 2 * index.float() - 1


    def rl(score_r):
        ###score_r : 300*1
        p_pos, p_neg = score_r, 1 - score_r
        # torch.cat的功能是将多个tensor类型矩阵的连接。它有两个参数，
        # 第一个是tensor元组或者tensor列表；第二个是dim，如果tensor是二维的，dim=0指在行上连接，dim=1指在列上连接
        p = torch.cat((p_neg, p_pos), 1)
        #### p : 300*2
        #输出形状中，将dim维设定为1，其它与输入形状保持一致。下面以列的形式输出p 中每行最大的元素
        #返回最大值，最大值的索引（从0开始）
        max_prob, max_index = torch.max(p, 1)
        ####max_prob : 300*1
        ####max_index : 300*1 (0/1)
        #float()将数据转换为浮点型
        #unsqueeze(1)返回一个新的张量，对输入的既定位置插入维度 1
        normal_index = max_index.float().unsqueeze(1)
        reverse_index = 1.0 - normal_index
        action_index = torch.cat((reverse_index, normal_index), 1)

        neg_log_prob = torch.log(max_prob)

        ###change to nll
        label = index2label(max_index)
        ####label: 300*1 (-1/1)
        # print('mmm',max_prob)
        # print('label',label)
        max_prob = max_prob * label
        # print('max_prob',max_prob.size())
        return max_prob.unsqueeze(1), action_index

    #通过format函数，找到对应的数据集文件
    cfg_path = './cfg/{}'.format(opt.dataset)
    #os.path.join 用于拼接路径，读取文件数，下面三个文件可以看做参数文件
    pn_cfg = read_cfg_file(os.path.join(cfg_path, 'PN'))
    upu_cfg = read_cfg_file(os.path.join(cfg_path, 'uPU'))
    nnpu_cfg = read_cfg_file(os.path.join(cfg_path, 'nnPU'))
    #这是一个检验函数，检验文件里的参数是否正确
    #get_unique_name()返回的是'{}'.format(int(time.time()))，返回当前时间
    assert \
        upu_cfg['dataset']['dataset_name'] == \
        nnpu_cfg['dataset']['dataset_name'] and \
        upu_cfg['network']['network_name'] == \
        nnpu_cfg['network']['network_name'] and \
        pn_cfg['dataset']['dataset_name'] == \
        nnpu_cfg['dataset']['dataset_name'] and \
        pn_cfg['network']['network_name'] == \
        nnpu_cfg['network']['network_name']
    exp_name = 'exp_{}_{}_{}'.format(
        nnpu_cfg['dataset']['dataset_name'],
        nnpu_cfg['network']['network_name'],
        get_unique_name()
    )
    #它实例化了一个叫LogData的类，位于helper文件夹的file_helper文件的最下面
    log_data = LogData()

    # upu and nnpu.
    # load_dataset 位于helper文件夹的training_helper文件
    # 实例化两个类，这两个类中的属性是处理后的数据集数据（归一化处理后的训练数据和测试数据）
    # 在这两个类中，下载了数据集并对数据进行了简单处理，以下三行信息量极大
    #完成了迭代器的初始化，挑选出了第一批训练数据和测试数据
    PuDataset, PnDataset = load_dataset(upu_cfg)
    pu_dataset = PuDataset(upu_cfg['dataset'])
    training_iterator = pu_dataset.get_training_iterator()

    # Loss function
    adversarial_loss = torch.nn.BCELoss()

    # Initialize generator and discriminator
    # generator = Generator()
    if model_style[opt.dataset] == 'mlp':
        discriminator = Discriminator_MLP()
        recognizer = Recognizer_MLP()
    else:
        discriminator = Discriminator_CNN()
        recognizer = Recognizer_CNN()

    if cuda:
        # generator.cuda()
        discriminator.cuda()
        recognizer.cuda()
        # adversarial_loss.cuda()

    # Optimizers
    # optimizer_G = torch.optim.Adam(generator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))
    optimizer = "Adam"
    optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2), weight_decay=1e-3)
    optimizer_R = torch.optim.Adam(recognizer.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2), weight_decay=1e-3)
    # data.cuda()就转换为GPU的张量类型，torch.cuda.FloatTensor类型
    Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
    TensorB = torch.cuda.ByteTensor if cuda else torch.ByteTensor

    # ----------
    #  Training
    # ----------
    epoch = 0
    i = 0
    eps = 1e-2
    gamma = 2.0  #gamma 参数可以看作是被模型选作支持向量的辐射范围的倒数
    beta = 10  # 10 for mnist
    epoch_num = 0
    #Precision从预测结果角度出发，描述了二分类器预测出来的正例结果中有多少是真实正例，
    # 即该二分类器预测的正例有多少是准确的
    # Precision（精准度），TP/(TP+FP)
    all_precision = []
    # Recall从真实结果角度出发，描述了测试集中的真实正例有多少被二分类器挑选了出来，
    # 即真实的正例有多少被该二分类器召回。
    #Recall=TP/(TP+FN)
    all_recall = []
    # F1得分是统计学中用来衡量二分类模型精确度的一种指标
    all_f1 = []
    all_acc = []
    big_acc = 100
    big_f = 0
    decrease_epoch = 0
    #猜测:创建结果目录，并以csv的格式储存结果
    if optimize_type == "holder":
        if ratio != 1:
            filepath = f"dataframe/{optimize_type}/{opt.dataset}_{loss_type}_{ratio}_{a}.csv"
        else:
            filepath = f"dataframe/{optimize_type}/{opt.dataset}_original_{a}.csv"
    if optimize_type == "early_stopping":
        filepath = f"dataframe/{optimize_type}/{opt.dataset}_{optimizer}.csv"
    if optimize_type == "batch":
        filepath = f"dataframe/{optimize_type}/{opt.dataset}_{upu_cfg.get('batch_size')}.csv"
    #以新建的方式（会对原有文件进行覆盖），创建文件
    f = open(filepath, "w")
    f.writelines("loss_d,loss_r,acc,precision,recall,f1\n")

    for imgs in training_iterator:
        #使用  torch.autograd.set_detect_anomaly(True)​​ 来帮助我们定位具体的出错位置
        with torch.autograd.set_detect_anomaly(True):

            # Adversarial ground truths
            #variable是一种可以不断变化的变量，符合反向传播，参数更新的属性。pytorch的variable是一个存放会变化值的地理位置，里面的值会不停变化，，
            # 、pytorch都是由tensor计算的，而tensor里面的参数是variable形式。
            imgs_data = Variable(Tensor(imgs[0]), requires_grad=False)
            # print(torch.max(imgs_data))
            imgs_label = Variable(Tensor(imgs[1]), requires_grad=False)
            positive_num = int(torch.sum(imgs_label).data)

            # imgs_u_data = Tensor(imgs[0][0])
            # imgs_u_label = Tensor(imgs[1][0])/
            # imgs_l_data = Tensor(imgs[0][1])
            # imgs_l_label = Tensor(imgs[1][1])

            # -----------------
            #  Train Generator
            # -----------------

            if opt.model == 'nnpu':
                score_D = discriminator.forward(imgs_data)
                score_R = recognizer.forward(imgs_data)
                #损失函数
                loss_p = 0.5 * torch.sum((1 - score_R) * imgs_label) / torch.sum(imgs_label)
                loss_u = torch.sum(score_R * (1 - imgs_label)) / torch.sum(1 - imgs_label) - 0.5 * torch.sum(
                    (score_R) * imgs_label) / torch.sum(imgs_label)

                objective = loss_p + loss_u
                if loss_u.data < -0:
                    objective = loss_p - 0
                    out = - loss_u
                else:
                    out = objective
                loss_max_d = objective
                loss_r = loss_u.clone()
                optimizer_R.zero_grad()
                out.backward()
                optimizer_R.step()

            if opt.model == 'upu':
                score_D = discriminator.forward(imgs_data)
                score_R = recognizer.forward(imgs_data)

                loss_p = 0.5 * torch.sum((1 - score_R) * imgs_label) / torch.sum(imgs_label)
                loss_u = torch.sum(score_R * (1 - imgs_label)) / torch.sum(1 - imgs_label) - 0.5 * torch.sum(
                    (score_R) * imgs_label) / torch.sum(imgs_label)

                objective = loss_p + loss_u
                # if loss_u.data < -0:
                #     objective = loss_p - 0
                #     out = - loss_u
                # else:
                out = objective
                loss_max_d = objective
                loss_r = loss_u.clone()
                #意思是把梯度置零，也就是把loss关于weight的导数变成0
                optimizer_R.zero_grad()
                #backward函数属于torch.autograd函数库，在深度学习过程中对函数进行反向传播，计算输出变量关于输入变量的梯度。
                out.backward()
                #执行单个优化步骤(参数更新)
                optimizer_R.step()

            if opt.model == 'pan':
                unlabeled_data = imgs_data[imgs_label.view(-1) == 0]

                score_R = recognizer.forward(unlabeled_data)
                score_D = discriminator.forward(imgs_data)
                # score_R = recognizer.forward(imgs_data)
                # 得到区分器discrimation 关于判断为正的得分（猜测）
                score_D_p = score_D[imgs_label.view(-1) == 1]
                score_D_u = score_D[imgs_label.view(-1) == 0]
                #这是一个对齐的作用，因为未标签数据的数量远大于标签数据的数量
                score_D_u_h = score_D_u[:score_D_p.shape[0]]
                score_R_h = score_R[:score_D_p.shape[0]]

                # top-k loss2 (sum)
                # kl = (1 * torch.log(1 - score_R + eps + 0.0) - torch.log(score_R + eps + 0.0)) * (2 * score_D_u - 1.0)
                # k = int(score_R.shape[0] * ratio)
                # kl, indices = torch.topk(kl, k, dim=0)

                # top-k KL(Du||Cu)
                # kl = score_D_u * torch.log(score_D_u / score_R) + (1 - score_D_u) * torch.log((1 - score_D_u) / (1 - score_R))

                # top-k KL(Du||1-Cu)
                #  KL散度计算公式（标记1）
                #先乘除后加减，有括号先计算括号里面的
                kl = score_D_u * torch.log(score_D_u / (1 - score_R)) + (1 - score_D_u) * torch.log((1 - score_D_u) / score_R)
                k = int(score_R.shape[0]*ratio)
                # 该函数的作用即按字面意思理解，topk:取数组的前k个元素进行排序。dim=0，代表从每一列取出K个元素
                # 通常该函数返回2个值，第一个值为排序的数组，第二个值为该数组中获取到的元素在原数组中的位置标号。
                kl, indices = torch.topk(kl, k, dim=0, sorted=False, largest=False)
                # kl, indices = torch.topk(kl, k, dim=0, sorted=False)
                # 根据index索引在input输入张量中选择某些特定的元素，在纵向的维度上
                score_R = torch.index_select(score_R, dim=0, index=indices.squeeze())
                score_D_u = torch.index_select(score_D_u, dim=0, index=indices.squeeze())
                # clone（）返回一个和源张量同shape、dtype和device的张量，与源张量不共享数据内存，但提供梯度的回溯
                #detach 意为分离，对某个张量调用函数 detach ()，detach() 的作用是返回一个 Tensor ，它和原张量的数据相同，但不具有梯度属性
                # 也就意味detach的方法，将variable参数从网络中隔离开，不参与参数更新
                R_rate = score_R.clone().detach()
                d_rate = score_D.clone().detach()
                d_rate_u = score_D_u_h.clone().detach()
                # 猜测
                w = 1.0 - torch.exp(- beta * (score_D - 1.0 / gamma) ** 4)
                #将一个tensor从创建它的图中分离，并把它设置成叶子tensor
                w.detach_()

                ######################################################
                ######################################################
                # please tunning the following hyper-params as different dataset may have different suitable hyper-params.
                ######################################################
                ######################################################

                # pdb.set_trace()
                # loss = imgs_label * torch.log(score_D + eps) + 0.01 * (1 - imgs_label) * (1 - R_rate) * torch.log( 1.0 - score_D + eps) + \
                #        0.0003 * w * (1 - imgs_label) * (1 - score_R + eps + 0.0) * torch.tan(1.5 * (gamma * d_rate - 1))
                # loss = imgs_label * torch.log(score_D + eps) + 0.01 * (1 - imgs_label) * (1 - R_rate) * torch.log( 1.0 - score_D + eps) + \
                #        0.0003 * w * (1 - imgs_label) * (1 - score_R + eps + 0.0) * torch.tan(1.5 * (gamma * d_rate - 1))
                # loss = imgs_label * torch.log(score_D + eps) + 0.000 * (1 - imgs_label) * torch.log(
                #    1.0 - score_D + eps) + w * 0.001 * (1 - imgs_label) * (1 * torch.log(1 - score_R + eps + 0.0) - torch.log(score_R + eps + 0.0)) * (score_D - torch.mean(d_rate))
                #猜测
                loss1 = torch.sum(torch.log(score_D_p + eps)) + 1.0 * torch.sum(
                    torch.max(torch.log(1.0 - score_D_u_h + eps) - torch.log(1.0 - torch.mean(d_rate_u)),
                              torch.zeros(d_rate_u.shape).cuda()))

                loss2 = 0.0001 * torch.sum((1 * torch.log(1 - score_R + eps + 0.0) - torch.log(score_R + eps + 0.0)) * (
                        2 * score_D_u - 1.0))  # torch.mean(d_rate_u)

                # loss2 = 0.0001 * torch.sum(kl)

                loss = loss1 + loss2

                # loss = imgs_label * torch.log(score_D + eps) + 0.02 * (1 - imgs_label) * torch.max(torch.log(1.0 - score_D + eps)-torch.log(1.0 - torch.mean(d_rate)), torch.zeros(d_rate.shape).cuda() - 10000.0) + w     * 0.001 * (1 - imgs_label) * (score_D * torch.log(score_D + eps) + (1 - score_D)* torch.log(1 - score_D + eps) - score_D * torch.log(score_R + eps) - (1 - score_D) * torch.log(1 - score_R + eps) )
                # loss = imgs_label * torch.log(score_D + eps) + 0.001 * (1 - imgs_label) * torch.log(
                #     1.0 - score_D + eps) + w * 0.0001 * (1 - imgs_label) * (1 * torch.log(1 - score_R + eps + 0.0) - torch.log(score_R + eps + 0.0)) * (score_D - torch.mean(d_rate))

                # for cifar
                # loss = imgs_label * torch.log(score_D + eps) + 0.05 * (1 - imgs_label) * torch.max(torch.log(1.0 - score_D + eps)-torch.log(1.0 - torch.mean(d_rate)), torch.zeros(d_rate.shape).cuda()) + w * 0.0001 * (1 - imgs_label) * (1 * torch.log(1 - score_R + eps + 0.0) - torch.log(score_R + eps + 0.0)) * (2 * score_D - 1.0)

                # for sdd
                # loss = imgs_label * torch.log(score_D + eps) + 0.1 * (1 - imgs_label) * torch.log(
                #     1.0 - score_D + eps) + w * 0.005 * (1 - imgs_label) * (1 * torch.log(1 - score_R + eps + 0.0) - torch.log(score_R + eps + 0.0)) * (score_D - torch.mean(d_rate))

                # loss = imgs_label * torch.log(score_D + eps) + 0.01 * (1 - imgs_label) * torch.log(
                #     1.0 - score_D + eps) + w * 0.0001 * (1 - imgs_label) * (
                #                    torch.log(1 - score_R + eps + 0.0) - 1.0 * torch.log(score_R + eps + 0.0)) * (
                #         score_D - torch.mean(d_rate))
                #
                # loss_1 =  imgs_label * (1.0 - 0.01 * score_R) * torch.log(score_D + eps) \

                # loss_2 = w * (1 - imgs_label) * (score_R + 10 * score_D) * torch.log(gamma - gamma * score_D + eps)
                # loss_2 =   w * (1 - imgs_label) * (score_R + 1.0 + eps + 0 * torch.log(score_D + eps)) * torch.log(gamma * score_D + eps)

                loss_max_d = - loss
                optimizer_D.zero_grad()
                loss_max_d.backward(retain_graph=True)
                optimizer_D.step()

                # ---------------------
                #  Train Recognizer
                # ---------------------
                score_R = recognizer.forward(unlabeled_data)
                score_D = discriminator.forward(imgs_data)
                # score_R = recognizer.forward(imgs_data)
                score_D_p = score_D[imgs_label.view(-1) == 1]
                score_D_u = score_D[imgs_label.view(-1) == 0]
                score_D_u_h = score_D_u[:score_D_p.shape[0]]
                score_R_h = score_R[:score_D_p.shape[0]]

                # top-k loss2
                # kl = (1 * torch.log(1 - score_R + eps + 0.0) - torch.log(score_R + eps + 0.0)) * (2 * score_D_u - 1.0)
                # k = int(score_R.shape[0] * ratio)
                # kl, indices = torch.topk(kl, k, dim=0)

                # top-k KL(Du||Cu)
                # kl = score_D_u * torch.log(score_D_u / score_R) + (1 - score_D_u) * torch.log((1 - score_D_u) / (1 - score_R))

                # top-k KL(Du||1-Cu)
                #KL散度（标记2）
                kl = score_D_u * torch.log(score_D_u / (1 - score_R)) + (1 - score_D_u) * torch.log((1 - score_D_u) / score_R)
                k = int(score_R.shape[0]*ratio)
                kl, indices = torch.topk(kl, k, dim=0, sorted=False, largest=False)
                # kl, indices = torch.topk(kl, k, dim=0, sorted=False)
                score_R = torch.index_select(score_R, dim=0, index=indices.squeeze())
                score_D_u = torch.index_select(score_D_u, dim=0, index=indices.squeeze())

                R_rate = score_R.clone().detach()
                d_rate = score_D.clone().detach()
                d_rate_u = score_D_u_h.clone().detach()
                #猜测
                loss1 = torch.sum(torch.log(score_D_p + eps)) + 1.0 * torch.sum(
                    torch.max(torch.log(1.0 - score_D_u_h + eps) - torch.log(1.0 - torch.mean(d_rate_u)),
                              torch.zeros(d_rate_u.shape).cuda()))

                loss2 = 0.0001 * torch.sum((1 * torch.log(1 - score_R + eps + 0.0) - torch.log(score_R + eps + 0.0)) * (
                            2 * score_D_u - 1.0))  # torch.mean(d_rate_u)

                # loss2 = 0.0001 * torch.sum(kl)

                loss = loss1 + loss2
                loss_r = loss.clone()
                optimizer_R.zero_grad()
                loss_r.backward()
                optimizer_R.step()

            if opt.model == 'agan':
                score_D = discriminator.forward(imgs_data)
                # 对score_D 内的所有元素，取平均值
                baseline = torch.mean(score_D).data
                # score_R.detach_()
                w = 1.0 - torch.exp(- beta * (score_D - 1.0 / gamma) ** 4)
                w.detach_()


                def cal_co_rl(epoch_):
                    if epoch_ > 10:
                        return 0.1
                    else:
                        return 0.0


                rl_loss_rate = cal_co_rl(epoch_num)

                rl_prob, action_index = rl(score_R)
                normal_reward = baseline - score_D
                reverse_reward = score_D - baseline

                reward = torch.cat((normal_reward, reverse_reward), 1)
                reward = torch.sum(reward * action_index, 1).unsqueeze(1)

                # pdb.set_trace()
                # rl_loss = torch.sum( rl_prob *  torch.log(gamma - gamma * score_D + eps) * rl_loss_rate )
                #猜测
                rl_loss = torch.sum(rl_prob * reward) * rl_loss_rate
                ptd_loss = torch.sum(imgs_label * torch.log(score_D + eps)) + torch.sum(
                    (1 - imgs_label[:positive_num]) * torch.log(1 - score_D[:positive_num] + eps))
                # ptd_loss = torch.mean( (1-imgs_label) * torch.log(1-score_D + eps) + imgs_label * torch.log(score_D + eps))
                # print(ptd_loss.size(),rl_loss.size(),type(rl_loss))
                loss = ptd_loss + rl_loss
                loss_max_d = - loss
                if epoch_num < 2000:
                    optimizer_D.zero_grad()
                    #retain_graph=True计算梯度所必要的buffer在经历过一次backward过程后不会被释放。
                    # 如果你想多次计算某个子图的梯度的时候，设置为True。
                    loss_max_d.backward(retain_graph=True)
                    optimizer_D.step()

                # ---------------------
                #  Train Recognizer
                # ---------------------

                optimizer_R.zero_grad()

                # loss *= 0.001

                loss.backward()
                # print(torch.sum(list(recognizer.parameters())[0].grad))
                optimizer_R.step()

            if training_iterator._finished_epoch != epoch:
                epoch_num += 1
                # adjust_lr(optimizer_D, epoch_num)
                # adjust_lr(optimizer_R, epoch_num)
                epoch = training_iterator._finished_epoch
                i = 0
                testiterator = pu_dataset.get_testing_iterator()
                correct = 0.0
                all = 0.0
                #返回传入字符串的表达式的结果。就是说：将字符串当成有效的表达式 来求值 并 返回计算结果。
                recognizer.eval()
                # discriminator.eval()
                TP = 0.0
                FP = 0.0
                FN = 0.0
                for test_imgs in testiterator:
                    test_imgs_data = Variable(Tensor(test_imgs[0]), requires_grad=False)
                    test_imgs_label = Variable(TensorB(test_imgs[1]), requires_grad=False)
                    # score = discriminator.forward(test_imgs_data)
                    score = recognizer.forward(test_imgs_data)
                    # pdb.set_trace()
                    #gt(a,b)函数比较a中元素大于（这里是严格大于）b中对应元素，大于则为1，不大于则为0，这里a为Tensor
                    #bytes() 函数返回字节对象。它可以将对象转换为字节对象，或创建指定大小的空字节对象
                    label_predicted = score.gt(0.5).byte() != test_imgs_label
                    TP += torch.sum(torch.Tensor.float((score[test_imgs_label == 1] >= 0.5).data))
                    FP += torch.sum(torch.Tensor.float((score[test_imgs_label == 0] >= 0.5).data))
                    FN += torch.sum(torch.Tensor.float((score[test_imgs_label == 1] <= 0.5).data))
                    # all_neg_score.append(score[test_imgs_label < 0.5].cpu().data.numpy())

                    label_predicted = torch.Tensor.float(label_predicted.data)
                    correct += torch.sum(label_predicted)
                    all += float(label_predicted.size(0))
                #设置模型为训练模式，即BatchNorm 层利用每个 batch 来统计，Dropout 层激活
                recognizer.train()

                precision = TP / (TP + FP + 1)
                recall = TP / (TP + FN + 1)
                # F1得分是统计学中用来衡量二分类模型精确度的一种指标
                f1 = 2 * precision * recall / (precision + recall + 1e-8)
                # acc = corrects / eval_num

                #####################################################

                # a , b = torch.max(score,1)
                acc = correct / all
                # print(acc)
                all_precision.append(precision)
                all_recall.append(recall)
                all_f1.append(f1)
                all_acc.append(acc)

                print("[Epoch %d/%d] [D loss: %f] [R loss: %f] [Accuracy: %f] [precision: %f] [recall: %f] [F1: %f]" % (
                    training_iterator._finished_epoch, opt.n_epochs,
                    loss_max_d, loss_r, 1 - acc, precision, recall, f1))

                f.writelines(f"{loss_max_d},{loss_r},{1 - acc},{precision},{recall},{f1}\n")

                if acc < big_acc:
                    big_acc = acc
                    decrease_epoch = 0
                if f1 > big_f:
                    big_f = f1
                    decrease_epoch = 0

                if acc - big_acc >= 0.01 and big_f - f1 >= 0.01:
                    decrease_epoch += 1

                if decrease_epoch >= 15:
                    break
            # batches_done = training_iterator._finished_epoch * training_iterator._len + i
            i = i + 1
            # if batches_done % opt.sample_interval == 0:
    print(loss_type, ratio)
    print("acc:", 1 - big_acc)
    print("f:", big_f)
    print("-")
    f.close()

    #     save_image(gen_imgs.data[:25], 'images/%d.png' % batches_done, nrow=5, normalize=True)
    # result = {'precision': all_precision,
    #           'recall': all_recall,
    #           'f1': all_f1,
    #           'acc': all_acc}
    # torch.save(result, opt.dataset + opt.model + '.pkl')
